In [41]:
from src.system.certificate.RASM import ReachAvoidSuperMartingaleCertificate
from src.system.polyhorn_helper import CommunicationBridge
from system.toolIO import Parser

# parser = Parser(
#     "./kaushik_sample/actions.json",
#     "./kaushik_sample/states.json",
#     "./kaushik_sample/dynamics.json",
#     "./kaushik_sample/noise.json",
#     "./kaushik_sample/config.json",
# )
parser = Parser(
    "./kaushik_sample_linear/actions.json",
    "./kaushik_sample_linear/states.json",
    "./kaushik_sample_linear/dynamics.json",
    "./kaushik_sample_linear/noise.json",
    "./kaushik_sample_linear/config.json",
)
tool_input = parser.parse()
print(tool_input)

ToolInput(state_space=Space(dimension=1, inequalities='-9<=S1<=10', listed_space_inequalities=None), action_policy=SystemControlPolicy(action_space=Space(dimension=1, inequalities='0<=A1<=1', listed_space_inequalities=None), state_dimension=1, maximal_degree=6, transitions=[Equation(monomials=[])], verification_mode=True, generated_constants=set(), constant_founded=False), disturbance=SystemStochasticNoise(dimension=2, distribution_name='normal', distribution_generator_parameters={'mean': [0, 0], 'std_dev': [0.1, 0.1], 'seed': 42}, noise_generators=NormalNoiseGenerator(mean=[0, 0], std_dev=[0.1, 0.1], dimension=2, seed=42)), dynamics=SystemDynamics(state_dimension=1, action_dimension=1, disturbance_dimension=2, system_transformations=[Equation(monomials=[Monomial(coefficient=1.00000000000000, variable_generators=('S1',), power=(1,)), Monomial(coefficient=0.500000000000000, variable_generators=[], power=[])])]), initial_states=Space(dimension=1, inequalities='1<=S1<=2', listed_space_ine

In [42]:
RASM = ReachAvoidSuperMartingaleCertificate(
    maximal_degree=tool_input.synthesis_config.maximal_polynomial_degree,
    state_dimension=tool_input.state_space.dimension,
)

In [43]:
RASM.define_constraints(tool_input)

In [44]:
print(RASM)

RASM Certificate: 
+	V(x) = (V1) + (S1 * V2) + (S1**2 * V3) + (S1**3 * V4) + (S1**4 * V5) + (S1**5 * V6) + (S1**6 * V7) 
- Constraints:
+	(V1) + (S1 * V2) + (S1**2 * V3) + (S1**3 * V4) + (S1**4 * V5) + (S1**5 * V6) + (S1**6 * V7) >= 0;
 	forall (10) + (-1 * S1) >= 0 and (S1) + (9) >= 0
+	(1) + (-1 * V1) + (-1 * S1 * V2) + (-1 * S1**2 * V3) + (-1 * S1**3 * V4) + (-1 * S1**4 * V5) + (-1 *
 	S1**5 * V6) + (-1 * S1**6 * V7) >= 0; forall 1<=S1<=2
+	(V1) + (S1 * V2) + (S1**2 * V3) + (S1**3 * V4) + (S1**4 * V5) + (S1**5 * V6) + (S1**6 * V7) +
 	(-5.00000000000000) >= 0; forall -9<=S1<=-8
+	(0) + (0) + (0) + (0) + (0) + (0) + (0) + (-3.00000000000000 * S1**5 * V7) + (-2.50 * S1**4 * V6) +
 	(-3.75 * S1**4 * V7) + (-2.00000000000000 * S1**3 * V5) + (-2.50 * S1**3 * V6) + (-2.50 * S1**3 *
 	V7) + (-1.50 * S1**2 * V4) + (-1.50 * S1**2 * V5) + (-1.25 * S1**2 * V6) + (-0.94 * S1**2 * V7) +
 	(-1.00000000000000 * S1 * V3) + (-0.75 * S1 * V4) + (-0.50 * S1 * V5) + (-0.31 * S1 * V6) + (-0.19
 	* S1 * 

In [45]:
for i, constraint in enumerate(RASM.get_constraints(), start=1):
    print(f"+ Constraint {i}:")
    print(" "*4, constraint.to_polyhorn_preorder())
    print()

+ Constraint 1:
     (assert (forall ((S1 Real)) (=> (and (>= (+ 10 (* -1 S1)) 0) (>= (+ (* 1 S1) 9) 0)) (>= (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (* (* 1 (* S1 (* S1 S1))) V4)) (* (* 1 (* (* S1 S1) (* S1 S1))) V5)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) (* (* 1 (* (* S1 (* S1 S1)) (* S1 (* S1 S1)))) V7)) 0)) ))

+ Constraint 2:
     (assert (forall ((S1 Real)) (=> (and (>= (+ 2 (* -1 S1)) 0) (>= (+ (* 1 S1) -1) 0)) (>= (+ (+ (+ (+ (+ (+ (+ 1 (* -1 V1)) (* (* -1 S1) V2)) (* (* -1 (* S1 S1)) V3)) (* (* -1 (* S1 (* S1 S1))) V4)) (* (* -1 (* (* S1 S1) (* S1 S1))) V5)) (* (* -1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) (* (* -1 (* (* S1 (* S1 S1)) (* S1 (* S1 S1)))) V7)) 0)) ))

+ Constraint 3:
     (assert (forall ((S1 Real)) (=> (and (>= (+ -8 (* -1 S1)) 0) (>= (+ (* 1 S1) 9) 0)) (>= (+ (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (* (* 1 (* S1 (* S1 S1))) V4)) (* (* 1 (* (* S1 S1) (* S1 S1))) V5)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)

# Feed to Polyhorn

In [46]:
input_string = CommunicationBridge.get_input_string(
    tool_io=tool_input,
    certificate=RASM,
)
input_config = CommunicationBridge.get_input_config(
    tool_io=tool_input,
)

In [47]:
print(input_string)

(declare-const V5 Real)
(declare-const V6 Real)
(declare-const V2 Real)
(declare-const V1 Real)
(declare-const V7 Real)
(declare-const V3 Real)
(declare-const V4 Real)

(assert (forall ((S1 Real)) (=> (and (>= (+ 10 (* -1 S1)) 0) (>= (+ (* 1 S1) 9) 0)) (>= (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (* (* 1 (* S1 (* S1 S1))) V4)) (* (* 1 (* (* S1 S1) (* S1 S1))) V5)) (* (* 1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) (* (* 1 (* (* S1 (* S1 S1)) (* S1 (* S1 S1)))) V7)) 0)) ))
(assert (forall ((S1 Real)) (=> (and (>= (+ 2 (* -1 S1)) 0) (>= (+ (* 1 S1) -1) 0)) (>= (+ (+ (+ (+ (+ (+ (+ 1 (* -1 V1)) (* (* -1 S1) V2)) (* (* -1 (* S1 S1)) V3)) (* (* -1 (* S1 (* S1 S1))) V4)) (* (* -1 (* (* S1 S1) (* S1 S1))) V5)) (* (* -1 (* (* S1 S1) (* S1 (* S1 S1)))) V6)) (* (* -1 (* (* S1 (* S1 S1)) (* S1 (* S1 S1)))) V7)) 0)) ))
(assert (forall ((S1 Real)) (=> (and (>= (+ -8 (* -1 S1)) 0) (>= (+ (* 1 S1) 9) 0)) (>= (+ (+ (+ (+ (+ (+ (+ (* 1 V1) (* (* 1 S1) V2)) (* (* 1 (* S1 S1)) V3)) (

In [48]:
CommunicationBridge.feed_to_polyhorn(config=input_config, input_string=input_string)

Running solver...
Is SAT: sat
Model: {'V5': '(- (/ 68876999.0 871142073297.0))', 'V6': '(- (/ 197929.0 580761382198.0))', 'V2': '(- (/ 1461948908935.0 5226852439782.0))', 'V1': '(/ 3277511967925.0 2613426219891.0)', 'V7': '(/ 320497.0 5226852439782.0)', 'V3': '(/ 45236078345.0 1742284146594.0)', 'V4': '(- (/ 758049655.0 2613426219891.0))'}
